In [ ]:
!pip install roboflow

In [1]:
import torch
from ultralytics import YOLO

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="s3h3QivGiiyTS3XN7Hib")
project = rf.workspace("project-ehjby").project("faces-detection-uolqa")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
weights = [
    "yolov8m-face.pt",
    "yolov8n-face.pt",
    "yolov8l-face.pt",
    "yolov5m-face.pt",
    "yolov5n-face.pt",
    "yolov5l-face.pt",
]

for weight in weights:
    detection_model = YOLO(weight)
    detection_model = detection_model.to("cuda")

    metrics = detection_model.val(data="Faces-Detection-2/data.yaml")
    print(metrics.box.map50)
    print(metrics.box.map75)
    print(metrics.box.map)

In [12]:
metrics.box.map50, metrics.box.map75, metrics.box.map

(0.9856793261798135, 0.8364813297963953, 0.6537393275038478)

In [22]:
import pandas as pd

index = ["yolov8-m", "yolov8-n", "yolov8-l", "yolov5-m", "yolov5-n", "yolov5-l"]
final_metrics = pd.DataFrame({
    "map@50": [0.9821, 0.9826, 0.9857, 0.8213, 0.7966, 0.8301],
    "map@75": [0.8765, 0.8365, 0.8786, 0.7131, 0.6834, 0.7262],
    "map@50:95": [0.6850, 0.6537, 0.7048, 0.6306, 0.6019, 0.6893],
    "time, ms": [20.2, 3.1, 28.1, 9.9, 7.5, 15.9]
}, index=index)

final_metrics

,map@50,map@75,map@50:95,"time, ms"
yolov8-m,0.9821,0.8765,0.6850,20.2
yolov8-n,0.9826,0.8365,0.6537,3.1
yolov8-l,0.9857,0.8786,0.7048,28.1
yolov5-m,0.8213,0.7131,0.6306,9.9
yolov5-n,0.7966,0.6834,0.6019,7.5
yolov5-l,0.8301,0.7262,0.6893,15.9
